# Projet Analyse de Données

## Gini Clustering

- Hugo Abreu

## Imports

In [1]:
# General Libraries
import numpy as np
import matplotlib.pyplot as plt
import requests
import math
import random
import time
import pickle

# Visualisation libraries
from mpl_toolkits.mplot3d import Axes3D 
from matplotlib import animation
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import HTML

%matplotlib inline 

## Create the point distribution

Let's build a function to create the characteristics of the wanted distribution: we want to be able to specify the dimension, the number of modes (distinct zones to be classified) and the parameters of the gaussian distribution determining the dispertion of points in each mode "zone".

- This function shows a prompt where you can specify the characteristics of the distribution

In [2]:
def distribution():
    dim = int(input("data dimension: "))
    md = int(input("number of modes:"))

    distribution = []

    # for each mode
    for i in range(md):
        mu = []
        sigma = []
        print("\nfor mode {}:".format(i))

        # for each dimention ask mu
        for j in range(dim):
            mu.append(float(input("mu for dim {}:".format(j))))

        # for each dimention ask sigma
        for k in range(dim):
            sigma.append(float(input("sigma for dim {}:".format(k))))

        # ask number of points for the current mode
        numpts = int(input("number of points:"))
        distribution.append([mu,sigma,numpts])

    return distribution, dim

We also need a function to output randomized points according to the distribution specifications. 

We are going to use a function to output randomized point coordinates in each dimention according to a gaussian distribution (`gauss()`).

Then, we can use a function `rddata()` that inputs the distribution specifications given by `distribution()` and uses `gauss()` to create the point dictionnary.

  I chose a dictionnary since you can easily access it by the coordinates of the points (the key), and you can easily remove entres. Contrary to a ndarray, the first data structure I chose, you can have a different amount of points in each mode.

In [3]:
def gauss(mu,sigma,dim):
    res = ()

    # for each dimention, create a point coordinate given gaussian distribution parameters
    for i in range(dim):
        res = res + (random.gauss(mu[i], sigma[i]),)
    return res

In [4]:
def rddata(distributions,dim):
    pts = {}
    modes = []

    # for each entry in distribution, create a mode specific point distribution
    for dist, params in enumerate(distributions):
        # for each mode, create points
        for nbpoints in range(params[2]):
            pt = gauss(params[0],params[1],dim)
            pts[pt]=dist
        modes.append(dist)

    # maximum coordinates in all dimentions (to be able to correctly plot)
    scale = []
    for d in range(dim):
        init = next(iter(pts))[d]
        scale.append([init,init])

        for i in pts:
            if i[d] < scale[d][0]:
                scale[d][0] = i[d]
            if i[d] > scale[d][1]:
                scale[d][1] = i[d]

    return pts, modes, scale

If we want to manually input points, they should be in the following format:

In [5]:
example_dict = {('point coordinates, separated by comma'): 'mode'}

In [6]:
example_dict

{'point coordinates, separated by comma': 'mode'}

### Example:

lets create a data distribution and generate a dictionnary of random points correponding to that distribution.

In [7]:
dist,dim = distribution()

data dimension:  3
number of modes: 5



for mode 0:


mu for dim 0: -1
mu for dim 1: -1
mu for dim 2: -1
sigma for dim 0: 0.5
sigma for dim 1: 0.5
sigma for dim 2: 0.5
number of points: 20



for mode 1:


mu for dim 0: -1
mu for dim 1: -2
mu for dim 2: -2
sigma for dim 0: 0.5
sigma for dim 1: 0.5
sigma for dim 2: 0.5
number of points: 25


...

mu for dim 0: -3
mu for dim 1: -1
mu for dim 2: -5
sigma for dim 0: 0.5
sigma for dim 1: 0.5
sigma for dim 2: 0.5
number of points: 24



for mode 3:


mu for dim 0: 2
mu for dim 1: 0
mu for dim 2: 4
sigma for dim 0: 0.5
sigma for dim 1: 0.5
sigma for dim 2: 0.5
number of points: 21



for mode 4:


mu for dim 0: 0
mu for dim 1: 5
mu for dim 2: 5
sigma for dim 0: 0.5
sigma for dim 1: 0.5
sigma for dim 2: 0.5
number of points: 27


let's save dist to a file so that the rest of the program can be tested without manually inputing distribution parameters

In [11]:
with open("distribution.txt", "wb") as distribution_file:
    pickle.dump(dist, distribution_file)

In [7]:
with open("distribution.txt", "rb") as distribution_file:
    distribution_from_file = pickle.load(distribution_file)

In [8]:
distribution_from_file

[[[-1.0, -1.0, -1.0], [0.5, 0.5, 0.5], 20],
 [[-1.0, -2.0, -2.0], [0.5, 0.5, 0.5], 25],
 [[-3.0, -1.0, -5.0], [0.5, 0.5, 0.5], 24],
 [[2.0, 0.0, 4.0], [0.5, 0.5, 0.5], 21],
 [[0.0, 5.0, 5.0], [0.5, 0.5, 0.5], 27]]

with this distribution file, you need to use dim equal to 3:

In [9]:
dim = 3

In [12]:
dist = distribution_from_file

In [13]:
pts, modes, scale = rddata(dist,dim)

Let's build a function to visualize these points: in this case, they are in 3 dimensions. I will build functions to visualize points in 2 and 3 dimensions, but all the other functions should work in every dimension.

## Display 

For 2d points, it is easy (the use of a dictionnary makes it simple to seperate colors according to modes)

In [14]:
def display2dpts(pts,modes):
    for x in modes:
        var = [coord for coord,mode in pts.items() if mode == x]
        plt.scatter(*zip(*var),edgecolors='black')

For 3d points, it is more difficult to have a clear view of the point distribution (since they can overlap). Let's build a function that outputs an animation of the plot, by stitching together plots with different viewing angles.

This function will also display partitions (useful for later in the project), but if we don't input a `partitions` parameter, it will only display the points.

- note: this function takes quite a lot of time run (30 seconds per animation approximately) since I manually render each frame

In [15]:
def display3dpts(pts,modes,scale, partitions_raw = False):
    ''' Display the point distribution in 3D'''
    fig = plt.figure()
    figct = 1

    ax = Axes3D(fig)

    # set borders
    ax.set_xlim3d(math.floor(scale[0][0]),math.ceil(scale[0][1]))
    ax.set_ylim3d(math.floor(scale[1][0]),math.ceil(scale[1][1]))
    ax.set_zlim3d(math.floor(scale[2][0]),math.ceil(scale[2][1]))


    def init():                 # build plot for each frame
        for x in modes:
            var = [coord for coord,mode in pts.items() if mode == x]
            if var != []:
                ax.scatter(*zip(*var),edgecolors='black')
        return fig,


    def animate(i):             # define view point function
        ax.view_init(elev=10., azim=i)
        return fig,


    # create animation
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                   frames=360, interval=20, blit=True)

    # save animation
    anim.save('fig%s.gif' % figct, writer=PillowWriter(fps=30))
    plt.close(fig)

    # display animation
    display(HTML('<img src="./fig%s.gif">' % figct))


    ''' Partition display (thresholds of the binary tree) '''
    if partitions_raw:
        partitions = []

        # for each recorded partition
        for t,td in partitions_raw:
            partitions.append([t,td])  # draw the plot with one more
                                       # partition than before
            fig = plt.figure()
            figct += 1

            ax = Axes3D(fig)

            # set borders
            ax.set_xlim3d(math.floor(scale[0][0]),math.ceil(scale[0][1]))
            ax.set_ylim3d(math.floor(scale[1][0]),math.ceil(scale[1][1]))
            ax.set_zlim3d(math.floor(scale[2][0]),math.ceil(scale[2][1]))

            def init():         # build plot for each frame
                # add point distribution
                for x in modes:
                    var = [coord for coord,mode in pts.items() if mode == x]
                    if var != []:
                        ax.scatter(*zip(*var),edgecolors='black')

                # add threshold representations
                for threshold, threshold_dim in partitions:
                    if threshold_dim == 0:  # thresholds for dim 0
                        y = np.arange(scale[1][0],scale[1][1],0.08)
                        z = np.arange(scale[2][0],scale[2][1],0.08)
                        Y,Z = np.meshgrid(y,z)

                        X = Y*0 + threshold

                        ax.plot_wireframe(X, Y, Z, rstride=10, cstride=10,color = 'coral', alpha=0.75)

                    elif threshold_dim == 1:  # thresholds for dim 1
                        x = np.arange(scale[0][0],scale[0][1],0.08)
                        z = np.arange(scale[2][0],scale[2][1],0.08)
                        X,Z = np.meshgrid(x,z)

                        Y = X*0 + threshold

                        ax.plot_wireframe(X, Y, Z, rstride=10, cstride=10, color = 'yellowgreen', alpha=0.75)

                    elif threshold_dim == 2:  # threshold for dim 3
                        x = np.arange(scale[0][0],scale[0][1],0.08)
                        y = np.arange(scale[1][0],scale[1][1],0.08)
                        X,Y = np.meshgrid(x,y)

                        Z = Y*0 + threshold

                        ax.plot_wireframe(X, Y, Z, rstride=10, cstride=10, color = 'teal', alpha=0.75)

                return fig,

            # animate plot
            anim = animation.FuncAnimation(fig, animate, init_func=init,
                                           frames=360, interval=20, blit=True)

            anim.save('fig%s.gif' % figct, writer=PillowWriter(fps=30))
            plt.close(fig)

            # display animation
            display(HTML('<img src="./fig%s.gif">' % figct))

In [16]:
display3dpts(pts,modes,scale)

## Gini: find a binary cut minimizing total gini

In each step, we want to create 

In [17]:
def gini(pts,modes):
    '''Compute gini for a given point distribution'''

    elements = []

    # compute how many elements of each mode is in pts
    for m in modes:
        elements.append(len([coord for coord,mode in pts.items() if mode == m]))

    # intermediate computations
    total = sum(elements)
    squares = [(x**2)/(total**2) for x in elements]

    # gini
    gini = 1 - np.sum(squares)

    return gini

In [18]:
def cut(pts,line):
    '''split a point distribution along a given line (threshold, dimention)'''

    threshold, dimention = line
    pts1,pts2 = {},{}

    # for each point
    for d in pts:
        # if point[dim] smaller than threshold, to the left
        if d[dimention]<=threshold:
            pts1[d] = pts[d]
        else:                   # else -> to the right
            pts2[d] = pts[d]
    return pts1,pts2

In [19]:
def optimize(pts,modes,dim,niter=150):
    '''optimize next cut'''

    # gini for given points
    ginipts = gini(pts,modes)

    # for a given number of iterations
    for iteration in range(niter):
        threshold_dim = np.random.randint(dim)  # pick a random dimention

        init = next(iter(pts))[threshold_dim]  # pick the first point
        threshold_scale = [init,init]          # initialize the scale

        # find the smallest and biggest point in the given dimension
        for i in pts:
            if i[threshold_dim] < threshold_scale[0]:
                threshold_scale[0] = i[threshold_dim]
            if i[threshold_dim] > threshold_scale[1]:
                threshold_scale[1] = i[threshold_dim]

        # choose the threshold according the scale (not to have meaningless thresholds)
        threshold = random.uniform(threshold_scale[0],threshold_scale[1])

        # build the line
        line = [threshold,threshold_dim]

        # split pts across the line
        pts1, pts2 = cut(pts,line)

        # compute gini
        ginipts1_norm = (len(pts1)/len(pts))*gini(pts1,modes)
        ginipts2_norm = (len(pts2)/len(pts))*gini(pts2,modes)

        # ginimin: value to maximize
        ginimin = ginipts - (ginipts1_norm + ginipts2_norm)

        if iteration:
            # test if previous ginimin smaller than current ginimin
            if ginimin > ginimin_opt:
                # if yes, keep it
                ginimin_opt, pts1_opt, pts2_opt, line_opt = ginimin, pts1, pts2, line
        else:
            # initialize
            ginimin_opt, pts1_opt, pts2_opt, line_opt = ginimin, pts1, pts2, line

    return ginimin_opt, pts1_opt, pts2_opt, line_opt

## Decision Tree

We now want to do it recursively for each node:

Let's create a `Node` class (python standard for manipulating binary trees). Each node will represent a set of points: as such, it will have the following parameters:
 
 - data dimention (`dim`)
 - list of existing modes (`modes`)
 - set of points, in the form of a dictionnary (`pts_dict`)

For each node, the following parameters can be retrieved in the previous information:
 - for each mode, the sum of points belonging to that mode (`elements`)
 - predicted class for the given set of points, given by the mode which has the most elements (`predicted_class`)
 - gini score for the given set of points (`gini`)
 
If the node isn't a leaf, meaning it's gini score isn't zero or maximum depth hasn't been reached, it will have left and right "sons" - resulting from the thresholding of the points according to one dimension. So nodes that aren't leaves will have the additional parameters:

 - threshold value (`threshold`)
 - threshold dimension (`threshold_dim`)
 - left son: another Node type object (`left`)
 - right son: another Node type object (`right`)

We can implement the previous functions inside the class

In [1]:
class Node:
    ###### INIT ######
    def __init__(self, dim, modes, pts_dict):
        # defining features of a node
        self.dim = dim
        self.modes = modes
        self.pts_dict = pts_dict
        self.elements = None
        self.predicted_class = None
        self.gini = None
        self.threshold = None
        self.threshold_dim = None
        self.left = None
        self.right = None

    ###### STATIC METHOD ######
    @staticmethod
    def _grow_tree(pts_dict, dim, modes, maxdepth, depth=0):
        # define a global variable partitions that we can later access to make decisions
        if depth==0:
            global partitions
            partitions = []

        # current node definition
        node = Node(
            dim = dim,
            modes = modes,
            pts_dict = pts_dict
        )

        node.elements = node._elements()
        node.predicted_class = node._predicted_class()
        node.gini = node._gini()

        # Split recursively, and define following nodes, until maximum depth is reached
        # or the gini value of the node is equal to 0
        if depth < maxdepth and node.gini != 0:
            ginimin_opt, pts_dict_left, pts_dict_right, line = node._optimize()

            node.threshold, node.threshold_dim = line

            partitions.append(line)

            node.left = Node._grow_tree(pts_dict_left, dim, modes, maxdepth, depth + 1)

            node.right = Node._grow_tree(pts_dict_right, dim, modes, maxdepth, depth + 1)
        return node

    ###### INSTANCE METHODS ###### 
    def _elements(self):
        '''pts per mode, for each mode, for given pts_dict'''

        elements = []
        for m in self.modes:
            elements.append(len([coord for coord,mode in self.pts_dict.items() if mode == m]))
        return elements

    #-------------------------#
    def _predicted_class(self):
        '''predicted mode for given thresholds'''
        return np.argmax(self.elements)

    #--------------------------#
    def _cut(self,line):
        '''split pts along a line (threshold, dim)'''
        threshold, dimention = line
        pts1,pts2 = {},{}
        for d in self.pts_dict:
            if d[dimention]<=threshold:
                pts1[d] = self.pts_dict[d]
            else:
                pts2[d] = self.pts_dict[d]
        return pts1,pts2

    #---------------------------#
    def _gini(self):
        '''compute gini for given points'''
        elements = []
        for m in self.modes:
            elements.append(len([coord for coord,mode in self.pts_dict.items() if mode == m]))

        total = sum(elements)
        squares = [(x**2)/(total**2) for x in elements]

        gini = 1 - np.sum(squares)

        return gini
    
    #----------------------------#
    def _optimize(self,niter=150):
        '''optimize next cut'''

        # gini for given points
        ginipts = self._gini()

        # for a given number of iterations
        for iteration in range(niter):
            threshold_dim = np.random.randint(self.dim)  # pick a random dimention

            init = next(iter(self.pts_dict))[threshold_dim]  # pick the first point
            threshold_scale = [init,init]                    # initialize the scale

            # find the smallest and biggest point in the given dimension
            for i in self.pts_dict:
                if i[threshold_dim] < threshold_scale[0]:
                    threshold_scale[0] = i[threshold_dim]
                if i[threshold_dim] > threshold_scale[1]:
                    threshold_scale[1] = i[threshold_dim]

            # choose the threshold according the scale (not to have meaningless thresholds)
            threshold = random.uniform(threshold_scale[0],threshold_scale[1])

            # build the line
            line = [threshold,threshold_dim]

            # split pts across the line
            pts1, pts2 = self._cut(line)

            # compute gini
            ginipts1_norm = (len(pts1)/len(self.pts_dict))*gini(pts1,self.modes)
            ginipts2_norm = (len(pts2)/len(self.pts_dict))*gini(pts2,self.modes)

            # ginimin: value to maximize
            ginimin = ginipts - (ginipts1_norm + ginipts2_norm)

            if iteration:
                # test if previous ginimin smaller than current ginimin
                if ginimin > ginimin_opt:
                    # if yes, keep it
                    ginimin_opt, pts1_opt, pts2_opt, line_opt = ginimin, pts1, pts2, line
            else:
                # initialize
                ginimin_opt, pts1_opt, pts2_opt, line_opt = ginimin, pts1, pts2, line

        return ginimin_opt, pts1_opt, pts2_opt, line_opt


    ###### DISPLAY ######

    def print_tree(self, pfx = "", pfx2 = ""):
        # if not end node, print threshold value, dim and predicted mode
        if self.threshold:
            print (pfx, "─ ", 'threshold value: %s, threshold dimention: %s, predicted mode: %s' 
                   % (self.threshold, self.threshold_dim, self.predicted_class))
            
        # if end node, print predicted mode and gini score
        else:
            print (pfx, "─ ", 'predicted mode: %s, gini score: %s' 
                   % (self.predicted_class, round(self.gini,2)))
            
        # if node has a left son, continue left, adding a left prefix while keeping previous prefixes
        if self.left != None:
            self.left.print_tree(pfx2 + "  │", pfx2 + "  │")
        
        # if node has a right son, continue right, adding a right prefix while keeping previous prefixes
        if self.right != None:
            self.right.print_tree(pfx2 + "  └", pfx2 + "   ")

In [21]:
tree = Node._grow_tree(pts, dim, modes, 50)

I implemented a method that prints the decision tree (Unix style):

In [22]:
tree.print_tree()

 ─  threshold value: 2.6888483275759065, threshold dimention: 1, predicted mode: 4
  │ ─  threshold value: -3.668879143179424, threshold dimention: 2, predicted mode: 1
  │  │ ─  predicted mode: 2, gini score: 0.0
  │  └ ─  threshold value: 1.7621171824360808, threshold dimention: 2, predicted mode: 1
  │     │ ─  threshold value: -1.1964082344713494, threshold dimention: 1, predicted mode: 1
  │     │  │ ─  threshold value: -1.0836341137230838, threshold dimention: 2, predicted mode: 1
  │     │  │  │ ─  threshold value: -1.470244573148615, threshold dimention: 2, predicted mode: 1
  │     │  │  │  │ ─  predicted mode: 1, gini score: 0.0
  │     │  │  │  └ ─  threshold value: -1.716021561591849, threshold dimention: 1, predicted mode: 1
  │     │  │  │     │ ─  predicted mode: 0, gini score: 0.0
  │     │  │  │     └ ─  predicted mode: 1, gini score: 0.0
  │     │  │  └ ─  threshold value: -2.2760178787873233, threshold dimention: 1, predicted mode: 0
  │     │  │     │ ─  predicted m

We can visualize the clustering process with my `display3dpts` function:

In [23]:
display3dpts(pts,modes,scale,partitions)

## Classify new points

In [24]:
def classify_pts(tree, pts):
    pts_class = []
    for pt in pts:
        node = tree

        while node.gini != 0:
            if pt[node.threshold_dim] < node.threshold:
                node = node.left
            else:
                node = node.right
        
        pts_class.append(node.predicted_class)
    return pts_class

If we create a simple distribution for testing:

In [25]:
dist_test, dim_test = distribution()

data dimension:  3
number of modes: 2



for mode 0:


mu for dim 0: -1
mu for dim 1: -2
mu for dim 2: -2
sigma for dim 0: 0.5
sigma for dim 1: 0.5
sigma for dim 2: 0.5
number of points: 5



for mode 1:


mu for dim 0: 0
mu for dim 1: 5
mu for dim 2: 5
sigma for dim 0: 0.5
sigma for dim 1: 0.5
sigma for dim 2: 0.5
number of points: 5


In [26]:
pts_test, modes_test, scale_test = rddata(dist_test, dim_test)

In [27]:
classify_pts(tree,pts_test)

[1, 1, 1, 1, 1, 4, 4, 4, 4, 4]

I chose mode 0 corresponding to mode 1 of the previously studied distribution, and mode 1 as mode 4 of the previosuly studied distribution.

All modes have been attributed correctly, my classification algorithm seems to have worked quite well! But with other random data in the same distribution center, results could vary if sigma is large...